# Tutorial PXCT data analysis (PART 2)- HERCULES school 2021

### Tutor: Julio C. da Silva (Néel Institute CNRS, Grenoble, France) 
### email: julio-cesar.da-silva@neel.cnrs.fr
#### Personal webpage: https://sites.google.com/view/jcesardasilva

### <span style="color:red">** Disclaimer: This notebook is intended from educational reasons only.**</span>
<span style="color:red">**Warning: You should have completed part 1 before starting part 2**</span>

<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://mybinder.org/v2/gh/jcesardasilva/tutorialHercules.git/HEAD"><img src="https://mybinder.org/static/images/logo_social.png" height="52" width="62"/>Run in MyBinder.org</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer" href="https://colab.research.google.com/github/jcesardasilva/tutorialHercules/blob/master/PXCT/PXCT_pipeline_part2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer"
href="https://jupyter-slurm.esrf.fr/"><img src="https://upload.wikimedia.org/wikipedia/fr/thumb/3/37/ESRF_-_Grenoble.png/280px-ESRF_-_Grenoble.png" height="40" width="40"/>Run in Jupyter-Slurm at ESRF (needs login)</a>
  </td>
    
  <td>
    <a target="_blank" rel="noopener noreferrer" href="https://github.com/jcesardasilva/tutorialHercules.git"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
   </td>
    
</table>

#### Importing packages again
Since we start a new notebook, we need to import the packages again:

In [ ]:
#%matplotlib ipympl
# standard packages
import time
import warnings
warnings.filterwarnings('ignore')
# third party packages
import ipympl
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import toupy

#### Let us reload our data 
We do this the same way we did in Part 1, but we only change the filename to `PXCTcorrprojections.npz`:

In [ ]:
fname = 'PXCTcorrprojections.npz'
data_dict = np.load(fname) # load the file
list(data_dict.files) # this one list the keys of the data dictionary extracted from the file
wavelen = data_dict['wavelen']
pixsize = data_dict['psize']
theta = data_dict['theta']
projections = data_dict['projections'].astype(np.float32) # <- ATTENTION: this one is memory consuming. 
nproj, nr, nc = projections.shape
delta_theta = np.diff(theta)

print(f"The total number of projections is {nproj}")
print(f"The angular sampling interval is {delta_theta[0]:.02f} degrees")
print(f"The projection pixel size of the projections is {pixsize/1e-9:.02f} nm")
print(f"The wavelenth of the incoming photons is {wavelen/1e-10:.02f} Angstroms")

Let us take a look at one projection. I will select the first and last ones,i.e. at angles 0 and 180-$\Delta\theta$ degress:

In [ ]:
plt.close('all')
fig1 = plt.figure(1,figsize=(10,4))
ax1 = fig1.add_subplot(121)
im1 = ax1.imshow(projections[0],cmap='bone',vmin=-4,vmax=1)
ax1.set_title('Phase proj. at 0 degrees',fontsize = 14)
#cax = divider.append_axes('right', size='5%', pad=0.05)
#fig1.colorbar(im1,cax=cax)
ax2 = fig1.add_subplot(122)
im2 = ax2.imshow(projections[-1],cmap='bone',vmin=-4,vmax=1)
ax2.set_title('Phase proj. at (180-0.4) degrees',fontsize = 14)
#display.display(fig1)

### Alignment of the tomographic projections
Great! Now that we have the projections corrected by the linear phase ramp and all unwrapped, we can start the alignment of the projections (registration in the language of digital signal processing). 

We will do it in the vertical and horizontal direction "independently", starting with the vertical alignment. 

#### Vertical alignment

The vertical alignment is performed based on the **Helgason-Ludwig consistency condition**. This is basically the Plancherel’s theorem for the Radon transform and states that the integral of any projection along horizontal directions is independent of the angle θ. 

In [ ]:
# import the toupy routines we will need
from toupy.registration import alignprojections_vertical

This step requires a certain number of parameters. For this reason, let us create a dictionary of parameters:

In [ ]:
params = dict() # initializing dictionary
params["pixtol"] = 0.1  # Tolerance of registration in pixels (E.g. 0.1 means 1/10 of pixel)
params["polyorder"] = 2  # Polynomial order to remove bias (E.g. 2 means 2nd order polynomial)
params["shiftmeth"] = "linear" # "linear" = bilinear interpolation or "fourier" = shift in the Fourier space
params["maxit"] = 10  # max of iterations
params["deltax"] = 20  # From edge of region to edge of image in x
params["limsy"] = (70, 200) # Vertical extend of the region to be considered in the alignment

Now that we have the parameters set, let us start the alignment. For we need to create an array which will keep the shifts to be applied to the projections in order to align them. This array, `shiftproj` which have a shape as `(2,nproj)`. Thus, `shiftproj[0]` will contain the shifts for the vertical diretions whereas `shiftproj[1]` will contains the shifts for the horizontal direction.

In [ ]:
# initializing shiftstack with zeros
shiftproj = np.zeros((2,nproj))
print(f"The shape of shifproj is {shiftproj.shape}")

In [ ]:
shiftproj, valignproj = alignprojections_vertical(projections,shiftproj,**params)

#### Let us take a look at the resulting alignment

In [ ]:
#-------
# you can put here the projections you want to display
disp = [300,329] # starts at index 0.
#-------
# preparing figure canvas
plt.close('all')
fig3 = plt.figure(3, figsize = (10,4))
ax31 = fig3.add_subplot(121)
im31 = ax31.imshow(projections[0],cmap='bone', vmin = -4, vmax = 1)
ax31.set_title('Projection number 1', fontsize = 16)
ax32 = fig3.add_subplot(122)
im32 = ax32.imshow(valignproj[0],cmap='bone', vmin = -4, vmax = 1)
ax32.set_title('Vert. aligned projec. number 1', fontsize = 16)
for ii in range(disp[0],disp[-1]+1):
    im31.set_data(projections[ii])
    ax31.set_title(f'Projection number {ii}', fontsize = 16)
    im32.set_data(valignproj[ii])
    ax32.set_title(f'Vert. aligned projec. number {ii}', fontsize = 16)
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
#release some memory
del projections

In [ ]:
sinogram = valignproj[:,300,:]
argangle = np.argsort(theta)
sinosort = sinogram[argangle]

# preparing figure canvas
plt.close('all')
fig4 = plt.figure(4, figsize = (12,4))
ax41 = fig4.add_subplot(121)
im41 = ax41.imshow(sinogram.T,cmap='bone', vmin = -4, vmax = 1)
ax41.set_xlabel('Projection number')
ax41.set_ylabel('Radial coordinate')
ax41.set_title('Original sinogram (interlaced)', fontsize = 16)
ax41.axis('tight')
ax42 = fig4.add_subplot(122)
im42 = ax42.imshow(sinosort.T,cmap='bone', vmin = -4, vmax = 1)
ax42.set_xlabel('Projection number')
ax42.set_ylabel('Radial coordinate')
ax42.set_title('Angle-sorted sinogram', fontsize = 16)
ax42.axis('tight')
display.display(plt.gcf())
display.clear_output(wait=True)

### Preparation for the horizontal alignment
For the horizontal alignment, we will use the tomographic consistency condition which reflects the uniqueness between the sinogram and the tomographic reconstructed slice, i.e., for each reconstructed slice, there is only one sinogram that can correspond to that reconstruction. 

Therefore, after reconstructing a slice and re-projecting to obtain the sinogram from the reconstructed slice, the resulting sinogram must be equal to the initial sinogram used for the reconstruction. If this is not the case, this will mean that the projections are horizontally misaligned.

Consequently, we need to be able to reconstruct a tomographic slice from the data we have so far. We will not do it using a non-standard tomographic reconstruction approach which is insensitive to spikes or possible phase wraps not yet corrected. For this, we will use the derivatives of the projections. 

#### Calculating the derivative of the projections

In [ ]:
# import the toupy routines we will need
from toupy.restoration import calculate_derivatives, chooseregiontoderivatives

Entering the required parameters

In [ ]:
params = dict() # initializing dictionary
params["deltax"] = 10  # From edge of region to edge of image in x
params["limsy"] = (55, 460)  # (top, bottom)
params["shift_method"] = "fourier" # "linear" = bilinear interpolation or "fourier" = shift in the Fourier space

In [ ]:
roix, roiy = chooseregiontoderivatives(valignproj, **params)

In [ ]:
valigndiff = calculate_derivatives(
        valignproj, roiy, roix, shift_method=params["shift_method"]
    )

Do you want to see how the derivatives of the projections look like?

In [ ]:
#------------
# parameters
#------------
# you can put here the projections you want to display
disp = [300,320] # starts at index 0.
#------------
# preparing figure canvas
plt.close('all')
fig3 = plt.figure(3, figsize = (10,4))
ax31 = fig3.add_subplot(121)
im31 = ax31.imshow(valignproj[0,roiy[0]:roiy[-1],roix[0]:roix[-1]],cmap='bone')
ax31.set_title('Projection number 1', fontsize = 16)
ax32 = fig3.add_subplot(122)
im32 = ax32.imshow(valigndiff[0],cmap='bone',vmin=-0.15, vmax=0.15)
ax32.set_title('Derivative of projec. number 1', fontsize = 16)
for ii in range(disp[0],disp[-1]+1):
    im31.set_data(valignproj[ii,roiy[0]:roiy[-1],roix[0]:roix[-1]])
    ax31.set_title(f'Projection number {ii}', fontsize = 16)
    im32.set_data(valigndiff[ii])
    ax32.set_title(f'Derivative of projec. number {ii}', fontsize = 16)
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
#release memory
del valignproj

#### Checking for the rotation axis positions
We will now visually check the rotation axis and get a prelimary approximation of its position. For this, we will now reconstruct our first tomographic slice, yet likely misaligned. For this reconstruction, we will use a modified Filtered Back Projection (FBP) algorithm which accepts the derivatives of the projections.

The tomographic reconstruction will be performed in CPU for the sake of this tutorial. But it can be much accelerated if implemented in GPU cards. 

The ESRF suite of software [`Nabu`](https://gitlab.esrf.fr/tomotools/nabu) and [`PyHST`](https://gitlab.esrf.fr/mirone/pyhst2) provide CUDA GPU-based reconstructions and are quite fast. The ESRF suite [`Silx`](https://github.com/silx-kit/silx) provides OpenCL-based reconstruction which is also fast in GPU cards.

In [ ]:
# import the toupy routines we will need
from toupy.registration import estimate_rot_axis

In [ ]:
params["slicenum"] = 225  # Choose the slice
params["filtertype"] = "hann"  # Filter to use for FBP
params["freqcutoff"] = 0.9  # Normalized frequency cutoff in case you want to apply a low pass band filter
params["circle"] = True # Apply a circular region in the external part of the slice
params["algorithm"] = "FBP" # Filtered Back projections algorithm
# initial guess of the offset of the axis of rotation
params["rot_axis_offset"] = 0
params["cliplow"] = None  # clip on low threshold for display
params["cliphigh"] = -1e-4  # clip on high threshold for display
params["sinohigh"] = None  # -0.1 # maximum gray level to display the sinograms
params["sinolow"] = None  # 0.1 # minimum gray level to display the sinograms
params["sinocmap"] = "bone" # sinogram colormap
params["colormap"] = "bone" # slice colormap
params["derivatives"] = True # True if the input is the derivative of the projections
params["calc_derivatives"] = False  # Calculate derivatives if not done

In [ ]:
estimate_rot_axis(valigndiff, theta, **params)

Now, we will start the **horizontal alignment**.

In [ ]:
# import the toupy routines we will need
from toupy.registration import (
    alignprojections_horizontal,
    compute_aligned_horizontal,
    oneslicefordisplay,
    refine_horizontalalignment,
    tomoconsistency_multiple,
)

I pasted some of the parameters below in order we can easily changed them if needed. We will already enter our prelimary estimate of the rotation axis position:

In [ ]:
params["slicenum"] = 225  # Choose the slice
params["filtertype"] = "hann"  # Filter to use for FBP
params["freqcutoff"] = 0.4  # Frequency cutoff (between 0 and 1)
params["circle"] = True # Apply a circular region in the external part of the slice
params["rot_axis_offset"] = 20 #<---- our estimate goes here --------
params["pixtol"] = 0.01  # Tolerance of registration in pixels
params["shiftmeth"] = "fourier"  # 'sinc' or 'linear' better for noise
params["maxit"] = 20  # max of iterations
params["cliplow"] = None  # clip air threshold
params["cliphigh"] = -4e-4  # clip on sample threshold
params["sinohigh"] = None
params["sinolow"] = None

We should remember now of our array `shiftproj` we created and used during the vertical alignment. We can see the vertical shifts are there, but the horizontal shifts are all 0. We will then add our preliminary estimate of the rotation axis offset to the horizonal shifts and, afterwards, we plot the `shiftproj` again:

In [ ]:
shiftproj[1] = np.zeros(valigndiff.shape[0]) # to be used in case of accidental overwritting
shiftproj_orig = shiftproj.copy() # keeping track of the original shiftproj

In [ ]:
shiftproj[1] = np.zeros(valigndiff.shape[0]) + params["rot_axis_offset"] # note we have reinforced the zero initialization to prevent accidents
fig4 = plt.figure(4,(10,4))
ax41 = fig4.add_subplot(121)
ax41.plot(shiftproj_orig.T)
ax41.legend(['vertical', 'horizontal'])
ax42 = fig4.add_subplot(122)
ax42.plot(shiftproj.T)
ax42.legend(['vertical', 'horizontal'])
display.display(plt.gcf())
display.clear_output(wait=True)

Great! Now we can begin the horizontal alignment procedure:

In [ ]:
# calculate the sinogram needed for the alignment
sinogram = np.transpose(valigndiff[:, params["slicenum"], :]).copy()
shiftproj = alignprojections_horizontal(sinogram, theta, shiftproj, **params)
# alignment refinement with different parameters if necessary
shiftstack, params = refine_horizontalalignment(
        valigndiff, theta, shiftproj, **params
)

Let us now repeat this alignment for 10 slices (-5 and +5 relative to the currently selected slicenum). At the end, we can decide to use (or not) the average of the shift values. 

In [ ]:
shiftproj = tomoconsistency_multiple(valigndiff, theta, shiftproj, **params)

Very good, we have reached a good alignment. Therefore, we not apply the shifts to the projections and reconstruct one tomographic slice for our inspection:

In [ ]:
alignedproj = compute_aligned_horizontal(
        valigndiff, shiftproj, shift_method=params["shiftmeth"]
    )

In [ ]:
# release some memory
del valigndiff

In [ ]:
# calculate one slice for display
aligned_sinogram = np.transpose(alignedproj[:, params["slicenum"], :])
oneslicefordisplay(aligned_sinogram, theta, **params)

#### Let us save our progress so far and make a break for discution/questions

In [ ]:
outputfname = "PXCTalignedprojections.npz"
np.savez(outputfname, wavelen = wavelen, psize = pixsize, projections = alignedproj, theta = theta)

In [ ]:
!ls -lrth